# Beta version
Blur only the face (and not a rectangle across the face)

In [1]:
import cv2
import numpy as np

from PIL import Image, ImageDraw
import face_recognition

In [ ]:
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    
    # Resize frame of video to 1/4 size for faster face detection processing
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Find all the faces and face encodings in the current frame of video
    face_locations = face_recognition.face_locations(small_frame)
    face_landmarks_list = face_recognition.face_landmarks(small_frame)
    

    # Create empty mask
    mask = np.zeros(small_frame.shape, dtype=np.uint8)
        
   # Create a PIL imagedraw object so we can draw on the picture
    pil_image = Image.fromarray(mask)
    d = ImageDraw.Draw(pil_image)
    
    # Draw mask
    for face_landmarks in face_landmarks_list:

        left_eyebrow_top = min(face_landmarks['left_eyebrow'], key = lambda t: t[1])
        right_eyebrow_top = min(face_landmarks['right_eyebrow'], key = lambda t: t[1])

        points = []
        points.insert(0, left_eyebrow_top)
        points.insert(0, right_eyebrow_top)
        points.insert(0, face_landmarks['chin'])

        cleaned = [j for i in points for j in i]

    d.polygon(cleaned, fill = 'white')
 
    # Apply mask
    blur = cv2.GaussianBlur(small_frame, (45,45), 30)
    blurred_img = np.where(d != 0, blur, small_frame)

    # Hit 'q' on the keyboard to quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

In [ ]:
video_capture.release()
cv2.destroyAllWindows()